In [1]:
%load_ext autoreload
%autoreload 2

In [214]:
%reload_ext autoreload

In [62]:
import logging
import logging.config
import configparser
import os
import time
from pathlib import Path
import signal
from cachepath import CachePath, TempPath, Path
import requests
import shutil



import lmsquery
from waveshare_epd import epd5in83

In [216]:
import cfg
import epdlib

In [5]:
# this works best as a global variable
logConfig = Path(cfg.LOGCONFIG)
logging.config.fileConfig(logConfig.absolute())
# logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(name)s %(levelname)s: %(message)s')

logger = logging.getLogger(__name__)

In [ ]:
appShortName = 'slimpi'
name = 'com.txoof.'
appLongName = name+appShortName

In [6]:
def configuration(configFile=None):
    
    configDefaults = cfg.CONFIGDEFAULTS
    
    if not configFile:
         configFile = Path(cfg.CONFIGFILE)

    config = configparser.ConfigParser()
    logger.info(f'reading configuration: {configFile}')
    config.read(configFile)
    
    try:
        for section in configDefaults:
            if section not in config.sections():
                logger.debug(f'adding section: {section}')
                config.add_section(section)
            
            for option in configDefaults[section]:
                if not config.has_option(section, option):
                    logger.debug(f'missing option: {option}')
                    logger.debug(f'setting {option} to: {configDefaults[section][option]}')
                    config[section][option] = str(configDefaults[section][option])
            with open(configFile, 'w') as file:
                config.write(file)
    
    except Exception as e:
        logging.exception(f'exception on configuration file: {e}')
        raise
    
    logger.debug(f'config file contains sections: {config.sections()}')
    return config

In [7]:
def query(lms, last=0, delay=7):
    '''query the player only when a specified delay has passed
    Accepts:
        last: float - last time query was called

    Returns:
        tuple(last, lms.now_playing()
    '''
    if last==0:
        last = time.clock_gettime(time.CLOCK_MONOTONIC)-delay

    if time.clock_gettime(time.CLOCK_MONOTONIC) > last+delay:
        return time.clock_gettime(time.CLOCK_MONOTONIC), lms.now_playing()

    else:
        return last, None

In [161]:
def cacheArt(query={}):
    if not query:
        return None
    
    cachePath = CachePath(appLongName, dir=True)
    required = ['artwork_url', 'album_id'] 
    missing = []
    
    for key in required:
        if not key in query:
            missing.append(key)
    if missing:
        logger.warning(f'required value(s) in `query` missing: {missing}')
        return None
    
    albumArtFile = cachePath/(query['album_id']+'.jpg')
    
    # check if file already exists in cache
    if albumArtFile.exists():
        logging.debug(f'album artwork already downloaded at: {albumArtFile}')
        return albumArtFile
    
    r = False
    albumArtURL = query['artwork_url']
    
    # try to fetch the album art
    try:
        r = requests.get(albumArtURL, stream=True)
    except requests.exceptions.RequestException as e:
        logging.error(f'failed to fetch artwork at {albumArtURL}: {e}')
        
    if r:
        try:
            with open(albumArtFile, 'wb') as outFile:
                shutil.copyfileobj(r.raw, outFile)
                logging.debug(f'wrote album art to: {albumArtFile}')
        except (OSError, FileExistsError, ValueError) as e:
            logging.error(f'failed to write {albumArtFile}: {e}')
    else:
        logging.error('skipping download of album artwork due to previous errors')

    return albumArtFile
    

In [8]:
class signalHandler(object):
    '''handle specific signals and allow graceful exiting while loop
    https://stackoverflow.com/a/31464349/5530152
    
    Signals Handled Gracefully:
        SIGINT
        SIGTERM
    Atributes:
        kill_now (bool) default: False
    '''
    kill_now = False
    def __init__(self):
        signal.signal(signal.SIGINT, self.exit_gracefully)
        signal.signal(signal.SIGTERM, self.exit_gracefully)
    
    def exit_gracefully(self, signum, frame):
        self.kill_now = True

In [218]:
def main():
    '''main entry point
    '''   
    logger.setLevel(logging.DEBUG)
    logger.info('Starting program')
    config = configuration()
    # scan for lms server and use the first one (this may be a terrible idea for other people)
#     lmsServer = lmsquery.scanLMS()[0]
    
    # define LMS query object
#     lms = lmsquery.LMSQuery(lmsServer['host'], lmsServer['port'], config.get('server', 'player'))
    lms = lmsquery.LMSQuery(player_id=config.get('server', 'player'))
    
    screen = epdlib.Screen()
    screen.epd = epd5in83.EPD()
    screen.initEPD()
#     screen.clearEPD()
    layout = epdlib.Layout(layout=epdlib.layouts.threeRow)
    layout.update_contents(query(lms, 0)[1])
    screen.initEPD()
    screen.elements=layout.blocks.values()
#     screen.concat()
#     screen.writeEPD()    
    
    
    sigHandler = signalHandler()
    
    # last update
    updated = 0
    # id of currently playing track
    nowPlayingID = None
    # status of player 
    nowPlayingMode = None

    # now playing fields to display
    fields = ['title', 'album', 'artist', 'mode', 'artwork_url']
    
    
    print(f'pid: {os.getpid()}')
    try:
        while not sigHandler.kill_now:
            updated, value = query(lms=lms, last=updated, delay=3)
            # add some try: around this
            response = value
            # if there's a response, check to see if display needs an update
            if response: 
                # if the album ID changed the player state changed, update the display
                if response['id'] != nowPlayingID or response['mode'] != nowPlayingMode:
                    
                    albumArt = cacheArt(response)
                    
                    if not albumArt:
                        albumArt = cfg.NOIMAGE
                        
                    value['coverart'] = str(albumArt)
                    
                    layout.update_contents(value)
                    screen.initEPD()
                    screen.elements=layout.blocks.values()
                    screen.concat()
                    screen.writeEPD()                      
                    
                    nowPlayingID = response['id']
                    nowPlayingMode = response['mode']
                    for field in fields:
                        print(f'{field}: {response[field]}')
                    print('\n')
            time.sleep(0.5)
    finally:
        print("cleaning up")
        screen.initEPD()
        screen.clearEPD()
        return layout
    

In [ ]:
if __name__ == '__main__':
    
    foo = main()

__main__         5:     INFO - Starting program
__main__         9:     INFO - reading configuration: servercfg.ini
__main__         30:     DEBUG - config file contains sections: ['server', 'screen']
root         69:     DEBUG - e-Paper busy
root         72:     DEBUG - e-Paper busy release
root         172:     DEBUG - calculating values from layout for resolution (600, 448)
root         202:     DEBUG - ***title***
root         108:     DEBUG - checking key/values
root         113:     DEBUG - missing key: dimensions; adding and setting to None
root         209:     DEBUG - dimensions: (600, 179)
root         234:     DEBUG - has explict position
root         236:     DEBUG - abs_coordinates: (0, 0)
root         134:     DEBUG - calculating font size
root         135:     DEBUG - using font at path: /home/pi/src/slimpi_epd/fonts/Open_Sans/OpenSans-Regular.ttf
root         145:     DEBUG - target Y fontsize: 62.65
root         158:     DEBUG - fontsize: 59
root         202:     DEBUG

root         307:     DEBUG - formatted list:
 ['OutKast feat.', 'Killer [...]']
root         336:     DEBUG - text image dimensions: (183, 71)
root         361:     DEBUG - v-center image coordinates
root         365:     DEBUG - image coordinates (360, 189)
root         297:     DEBUG - ignoring block coverid
root         297:     DEBUG - ignoring block duration
root         297:     DEBUG - ignoring block album_id
root         297:     DEBUG - ignoring block genre
root         294:     DEBUG - updating block: album
root         307:     DEBUG - formatted list:
 ['Speakerboxxx /', 'The Love Below']
root         336:     DEBUG - text image dimensions: (220, 73)
root         361:     DEBUG - v-center image coordinates
root         365:     DEBUG - image coordinates (360, 277)
root         297:     DEBUG - ignoring block artwork_url
root         297:     DEBUG - ignoring block time
root         294:     DEBUG - updating block: mode
root         307:     DEBUG - formatted list:
 ['play']

root         83:     DEBUG - close 5V, Module enters 0 power consumption ...
title: If That's Alright
album: Still Feel Gone
artist: Uncle Tupelo
mode: play
artwork_url: http://192.168.178.9:9000/music/19900489/cover.jpg


root         36:     DEBUG - wrote album art to: /tmp/com.txoof.slimpi/5680.jpg
root         288:     DEBUG - updating blocks
root         297:     DEBUG - ignoring block id
root         294:     DEBUG - updating block: title
root         307:     DEBUG - formatted list:
 ['Misunderstood']
root         336:     DEBUG - text image dimensions: (418, 65)
root         348:     DEBUG - h-center line: Misunderstood
root         356:     DEBUG - h-center image coordinates
root         361:     DEBUG - v-center image coordinates
root         365:     DEBUG - image coordinates (91, 57)
root         294:     DEBUG - updating block: artist
root         307:     DEBUG - formatted list:
 ['Wilco']
root         336:     DEBUG - text image dimensions: (76, 33)
root         361:    

In [11]:
    sc = epdlib.Screen()
    sc.epd = epd5in83.EPD()
    sc.initEPD()
    sc.clearEPD()

root         69:     DEBUG - e-Paper busy
root         72:     DEBUG - e-Paper busy release
root         69:     DEBUG - e-Paper busy
root         72:     DEBUG - e-Paper busy release


True